In [ ]:
"""
Pull data on party affiliation from Statistics Sweden, plot it, forecast a subset of it with a VAR(2) model, 
and finally add the data to a MySQL database. 

Necessary steps to take before running the script: 
Create database. 
Create table called scb with field id as primary key. 

Dependencies: json, urllib, codecs, matplotlib, numpy, statsmodels, pandas, MySQLdb
""""

import json
import urllib
import codecs
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.vector_ar import var_model
import pandas as pd
import MySQLdb

# Specify the url and the data to pull
url = "http://api.scb.se/OV0104/v1/doris/sv/ssd/START/ME/ME0201/ME0201A/Vid10"

values = {
  "query": [
    {
      "code": "Parti",
      "selection": {
        "filter": "item",
        "values": [
          "c",
          "l",
          "m",
          "kd",
          "nyd",
          "s",
          "v",
          "mp",
          "SD",
          "övr"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "ME0201B1"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat"
  }
}

# Pull and format the data
req = urllib.request.Request(url, values)
data = json.dumps(values)
binary_data = data.encode('utf-8')
post_response = urllib.request.urlopen(req, binary_data)
post_data = post_response.read().decode("UTF-8")
json_data = json.loads(post_data)

# Extract the relevant data from the dataset and transform it to a useful form
dataset = json_data['dataset']
time = dataset['dimension']['Tid']['category']['index']
parties = dataset['dimension']['Parti']['category']['index']
values = dataset['value']

T = len(time)
n = len(parties)

parties_inv = {v: k for k, v in parties.items()}
time_inv = {v: k for k, v in time.items()}

t = list(time_inv.keys())

# Plotting! 
plt.gca().set_prop_cycle(plt.cycler('color', plt.cm.brg(np.linspace(0, 1, n))))

for i in range(n):
    plt.plot(t,values[i*T:(i+1)*T])

plt.xticks(t[::20], list(time_inv.values())[::20])
plt.legend(parties_inv.values(), ncol=5)
plt.ylabel("Percentage")
plt.xlabel("Time")

# Add data on the parties that do not contain None values to DataFrame for VAR modelling. 
# Uncomment the commented lines if you want to deal with "övriga" (misc) and None values. 
val_df = pd.DataFrame()
val_df["c"] = values[:T]
val_df["l"] = values[T:2*T]
val_df["m"] = values[2*T:3*T]
#val_df["kd"] = values[3*T:4*T]
#val_df["nyd"] = values[4*T:5*T]
val_df["s"] = values[5*T:6*T]
val_df["v"] = values[6*T:7*T]
#val_df["mp"] = values[7*T:8*T]
#val_df["sd"] = values[8*T:9*T]
#val_df["ovr"] = values[9*T:]

# Prepare DataFrame for VAR modelling by making the indices the 
dates = pd.Series()
for i in range(T):
    date = pd.Series(pd.datetime(int(list(time_inv.values())[i][:4]), 
                            int(list(time_inv.values())[i][5:]), 1))
    dates = dates.append(date)
val_df.index = dates

# VAR model! 
model = var_model.VAR(val_df)
results = model.fit(2)

# Connect to database, remember to fill in the fields
conn = MySQLdb.connect(host = "",
                       user = "",
                       passwd = "",
                       db = "")
cursor = conn.cursor()

# Add rows to SQL table, one for each poll
for val in time_inv.values():
    query = "ALTER TABLE scb ADD %s FLOAT" % (val)
    cursor.execute(query)

# Prepare the INSERT statement for inserting data into the rows created above
cols_list = ['id'] + list(time_inv.values())
query = "INSERT INTO scb ("
for i in range(len(cols_list)):
    query += cols_list[i]
    query += ","
query = query[:-1]
query += ") VALUES ("
query += '%s, ' * 89
query = query[:-2]
query += ");"

# Insert data into db and close connection
for i in range(n):
    val_list = [parties_inv[i]] + values[i*T:(i+1)*T]
    val_tpl = tuple(val_list)
    cursor.execute(query, val_tpl)
    conn.commit()

cursor.close()